# Parser tests

In [1]:
[global]
from dsc.dsc_parser import DSC_Script as ds
from dsc.utils import FormatError

def test_format(obj, *args):
    try:
        return obj(*args)
    except Exception as e:
        assert type(e).__name__ == 'FormatError'
        print(e)
        return None

## Basic syntax

In [2]:
%run
[1]
# F: missing output
text = '''
simulate: sim.R
    x: 1
    y: 2
'''
test_format(ds, text)

Cannot find required section ``DSC``!


In [ ]:
%run
[2]
# F: grouped parameters is not allowed
text = '''
simulate: sim.R
    group_1:
        x: 1
        y: 2
DSC:
    run: simulate
'''
test_format(ds, text)

In [18]:
%run
[3]
# P: exec decoration
text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
DSC:
    run: simulate
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1]), ('y', [2])]

In [2]:
%run
[4]
# F: grouped parameters is not allowed in exec decoration
text = '''
simulate: sim.R
    @simulate:
        group_1:
            x: 1
            y: 2
DSC:
    run: simulate
'''
res = test_format(ds, text)

Variable grouping ``group_1`` is not allowed


In [1]:
%run
[5]
# F: alias is not a list
text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
        @ALIAS:
            x_1: x
DSC:
    run: simulate
'''
res = test_format(ds, text)

Invalid @ALIAS format for module ``simulate`` (has to be formatted `alias = variable`).


In [4]:
%run
[6]
# P: alias for simulate module
text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
        @ALIAS: x_1 = x
DSC:
    run: simulate
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('y', [2]), ('x_1', [1])]